In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /home/jovyan/Main.sc

Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/3.4.4/chisel3_2.12-3.4.4.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-reflect/2.12.12/scala-reflect-2.12.12.pom
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3-macros_2.12/3.4.4/chisel3-macros_2.12-3.4.4.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.12/scala-library-2.12.12.pom
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3-core_2.12/3.4.4/chisel3-core_2.12-3.4.4.pom
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl_2.12/1.4.4/firrtl_2.12-1.4.4.pom
Downloaded https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.7.1/antlr4-runtime-4.7.1.pom
Downloaded https://repo1.maven.org/maven2/net/jcazevedo/moultingyaml_2.12/0.4.2/moultingyaml_2.12-0.4.2.pom
Downloaded https://repo1.maven.org/maven2/org/json4s/json4s-native_2.12/

Compiling /home/jovyan/Main.sc #2

path: String = "/home/jovyan/source/load-ivy.sc"

In [14]:
import chisel3._
import chisel3.util._

class Sha1IO(val DATA_WIDTH: Int = 32) extends Bundle {
  val inValid  = Input(Bool())
  val inWord   = Input(UInt(DATA_WIDTH.W))
  val lastWord = Input(Bool())

  val inReady  = Output(Bool())
  val outValid = Output(Bool())
  val outHash  = Output(UInt(160.W))
}

object Sha1States {
  val sIdle :: sLoad :: sExpand :: sRound :: sDone :: Nil = Enum(5)
}

object Sha1Helpers {
  def rotl(x: UInt, n: Int): UInt = {
    require(n >= 0 && n < 32)
    ((x << n) | (x >> (32 - n).U))(31, 0)
  }
}


import chisel3._

import chisel3.util._


defined class Sha1IO
defined object Sha1States
defined object Sha1Helpers

In [18]:
class Sha1Scheduler extends Module {
  val io = IO(new Bundle {
    val inWord   = Input(UInt(32.W))
    val loadWord = Input(Bool())
    val expand   = Input(Bool())
    val Wout     = Output(UInt(32.W))
  })

  val W = Reg(Vec(20, UInt(32.W)))  // toy expansion buffer
  val idx = RegInit(0.U(5.W))

  when(io.loadWord) {
    W(idx) := io.inWord
    idx := idx + 1.U
  }.elsewhen(io.expand) {
    val newWord = W(idx-3.U) ^ W(idx-8.U) ^ W(idx-14.U) ^ W(idx-16.U)
    W(idx) := Sha1Helpers.rotl(newWord, 1)
    idx := idx + 1.U
  }

  io.Wout := W(idx)
}


defined class Sha1Scheduler

In [19]:
class Sha1Round extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(32.W))
    val b = Input(UInt(32.W))
    val c = Input(UInt(32.W))
    val d = Input(UInt(32.W))
    val e = Input(UInt(32.W))
    val W = Input(UInt(32.W))
    val k = Input(UInt(32.W))
    val out = Output(Vec(5, UInt(32.W)))
  })

  val f = (io.b & io.c) | (~io.b & io.d)  // round 0–19
  val temp = Sha1Helpers.rotl(io.a, 5) + f + io.e + io.k + io.W

  io.out(0) := temp
  io.out(1) := io.a
  io.out(2) := Sha1Helpers.rotl(io.b, 30)
  io.out(3) := io.c
  io.out(4) := io.d
}


defined class Sha1Round

In [20]:
class Sha1Datapath extends Module {
  val io = IO(new Bundle {
    val inWord    = Input(UInt(32.W))
    val loadWord  = Input(Bool())
    val expand    = Input(Bool())
    val doRound   = Input(Bool())
    val resetHash = Input(Bool())
    val outHash   = Output(UInt(160.W))
  })

  val H = RegInit(VecInit(
    "h67452301".U(32.W), "hefcdab89".U(32.W),
    "h98badcfe".U(32.W), "h10325476".U(32.W),
    "hc3d2e1f0".U(32.W)
  ))

  val a = RegInit(H(0)); val b = RegInit(H(1))
  val c = RegInit(H(2)); val d = RegInit(H(3)); val e = RegInit(H(4))

  val sched = Module(new Sha1Scheduler)
  val round = Module(new Sha1Round)

  sched.io.inWord   := io.inWord
  sched.io.loadWord := io.loadWord
  sched.io.expand   := io.expand

  round.io.a := a; round.io.b := b
  round.io.c := c; round.io.d := d
  round.io.e := e; round.io.W := sched.io.Wout
  round.io.k := "h5A827999".U

  when(io.resetHash) {
    a := H(0); b := H(1); c := H(2); d := H(3); e := H(4)
  }.elsewhen(io.doRound) {
    val next = round.io.out
    a := next(0); b := next(1); c := next(2)
    d := next(3); e := next(4)
  }

  io.outHash := Cat(a, b, c, d, e)
}


defined class Sha1Datapath

In [9]:
class Sha1FSM extends Module {
  val io = IO(new Bundle {
    val inValid   = Input(Bool())
    val lastWord  = Input(Bool())
    val loadWord  = Output(Bool())
    val expand    = Output(Bool())
    val doRound   = Output(Bool())
    val resetHash = Output(Bool())
    val inReady   = Output(Bool())
    val outValid  = Output(Bool())
  })

  val state = RegInit(Sha1States.sIdle)
  io.loadWord := false.B; io.expand := false.B
  io.doRound := false.B; io.resetHash := false.B

  io.inReady := (state =/= Sha1States.sDone)
  io.outValid := (state === Sha1States.sDone)

  switch(state) {
    is(Sha1States.sIdle) {
      when(io.inValid) { io.loadWord := true.B; state := Sha1States.sLoad }
    }
    is(Sha1States.sLoad) {
      when(io.inValid && io.lastWord) { state := Sha1States.sExpand }
    }
    is(Sha1States.sExpand) {
      io.expand := true.B
      state := Sha1States.sRound
    }
    is(Sha1States.sRound) {
      io.doRound := true.B
      state := Sha1States.sDone
    }
    is(Sha1States.sDone) {
      io.resetHash := true.B
      state := Sha1States.sIdle
    }
  }
}


defined class Sha1FSM

In [21]:
import chisel3.tester._
import chisel3.tester.RawTester.test

class Sha1Top extends Module {
  val io = IO(new Sha1IO)

  val fsm = Module(new Sha1FSM)
  val dp  = Module(new Sha1Datapath)

  dp.io.inWord    := io.inWord
  dp.io.loadWord  := fsm.io.loadWord
  dp.io.expand    := fsm.io.expand
  dp.io.doRound   := fsm.io.doRound
  dp.io.resetHash := fsm.io.resetHash

  fsm.io.inValid  := io.inValid
  fsm.io.lastWord := io.lastWord

  io.inReady  := fsm.io.inReady
  io.outValid := fsm.io.outValid
  io.outHash  := dp.io.outHash
}

object Sha1RawTestHarness extends App {
  test(new Sha1Top) { dut =>
    val words = Seq(
      BigInt("01234567", 16),
      BigInt("89ABCDEF", 16),
      BigInt("FEDCBA98", 16)
    )

    println("Feeding words into SHA-1 (simplified):")
    for ((w, i) <- words.zipWithIndex) {
      dut.io.inValid.poke(true.B)
      dut.io.inWord.poke(w.U(32.W))
      dut.io.lastWord.poke((i == words.length - 1).B)
      dut.clock.step(1)
      dut.io.inValid.poke(false.B)
    }

    while (!dut.io.outValid.peek().litToBoolean) {
      dut.clock.step(1)
    }

    val hash = dut.io.outHash.peek().litValue
    println(f"SHA-1 digest (simplified) = 0x$hash%040X")
  }
}


import chisel3.tester._

import chisel3.tester.RawTester.test


defined class Sha1Top
defined object Sha1RawTestHarness

In [ ]:
object Sha1QuickTest extends App {
  test(new Sha1Top) { dut =>
    val words = Seq(
      BigInt("01234567", 16),  // word 0
      BigInt("89ABCDEF", 16),  // word 1
      BigInt("FEDCBA98", 16)   // word 2
    )

    println("=== SHA-1 Simplified Test ===")
    for ((w, i) <- words.zipWithIndex) {
      dut.io.inValid.poke(true.B)
      dut.io.inWord.poke(w.U(32.W))
      dut.io.lastWord.poke((i == words.length - 1).B)
      dut.clock.step(1)
      dut.io.inValid.poke(false.B)
    }

    while (!dut.io.outValid.peek().litToBoolean) {
      dut.clock.step(1)
    }

    val hash = dut.io.outHash.peek().litValue
    println(f"Digest (simplified core) = 0x$hash%040X")
  }
}


defined object Sha1QuickTest

In [23]:
Sha1QuickTest.main(Array())


Elaborating design...
Done elaborating.
=== SHA-1 Simplified Test ===
Digest (simplified core) = 0x9FB498B3674523017BF36AE298BADCFE10325476
test Sha1Top Success: 0 tests passed in 7 cycles in 0.085306 seconds 82.06 Hz
